In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
from src.inference import get_feature_store

In [4]:
from datetime import datetime, timedelta
import pandas as pd  

# Get the current datetime64[us, Etc/UTC]  
current_date = pd.Timestamp.now(tz='Etc/UTC')
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1)),write_options={"use_spark": True}
)
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]

2025-05-08 17:29:37,805 INFO: Initializing external client
2025-05-08 17:29:37,805 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-08 17:29:39,030 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1213633
Fetching data from 2025-04-09 21:29:37.802767+00:00 to 2025-05-08 20:29:37.802767+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (4.74s) 


In [5]:
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)

,pickup_hour,pickup_location_id,rides
0,2025-04-09 22:00:00+00:00,5187.03,0
1,2025-04-09 23:00:00+00:00,5187.03,0
2,2025-04-10 00:00:00+00:00,5187.03,0
3,2025-04-10 01:00:00+00:00,5187.03,0
4,2025-04-10 02:00:00+00:00,5187.03,0
...,...,...,...
63930,2025-05-07 18:00:00+00:00,JC116,6
63931,2025-05-07 19:00:00+00:00,JC116,8
63932,2025-05-07 20:00:00+00:00,JC116,0
63933,2025-05-07 21:00:00+00:00,JC116,0


In [6]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 63935 entries, 0 to 66214
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype                  
---  ------              --------------  -----                  
 0   pickup_hour         63935 non-null  datetime64[us, Etc/UTC]
 1   pickup_location_id  63935 non-null  object                 
 2   rides               63935 non-null  int32                  
dtypes: datetime64[us, Etc/UTC](1), int32(1), object(1)
memory usage: 1.7+ MB


In [7]:
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)

In [8]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 63935 entries, 0 to 66214
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   pickup_hour         63935 non-null  datetime64[us]
 1   pickup_location_id  63935 non-null  object        
 2   rides               63935 non-null  int32         
dtypes: datetime64[us](1), int32(1), object(1)
memory usage: 1.7+ MB


In [9]:
from src.data_utils import transform_ts_data_into_features

features = transform_ts_data_into_features(
    ts_data,
    window_size=24*28,
    step_size=23
)


In [10]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,1,0,0,0,1,0,1,1,0,0,...,3,0,0,1,0,1,2,2,JC034,2025-05-07 22:00:00
1,0,0,0,0,1,1,1,0,0,0,...,0,0,0,1,0,0,2,3,JC053,2025-05-07 22:00:00
2,0,0,0,0,0,0,0,0,1,1,...,1,4,0,1,0,0,0,0,JC059,2025-05-07 22:00:00
3,0,5,1,1,0,3,9,3,0,1,...,2,4,8,0,0,0,0,5,HB105,2025-05-07 22:00:00
4,0,0,0,1,0,4,11,0,0,5,...,0,0,0,0,0,0,1,0,JC002,2025-05-07 22:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,0,1,0,6,0,4,0,8,1,0,...,8,2,1,4,5,0,6,0,HB609,2025-05-07 22:00:00
91,0,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,JC077,2025-05-07 22:00:00
92,1,1,0,0,1,0,1,0,2,5,...,1,1,0,1,0,0,0,4,JC074,2025-05-07 22:00:00
93,1,3,0,0,0,0,1,1,0,1,...,7,0,8,0,2,1,3,0,HB611,2025-05-07 22:00:00


In [11]:
from src.inference import load_batch_of_features_from_store
current_date = pd.Timestamp.now(tz='Etc/UTC')
features = load_batch_of_features_from_store(current_date)

2025-05-08 17:29:49,028 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-08 17:29:49,036 INFO: Initializing external client
2025-05-08 17:29:49,037 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-08 17:29:49,897 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1213633
Fetching data from 2025-04-09 21:29:49.028956+00:00 to 2025-05-08 20:29:49.028956+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (3.85s) 


In [12]:
current_date

Timestamp('2025-05-08 21:29:49.028956+0000', tz='Etc/UTC')

In [13]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,5187.03,2025-05-07 22:00:00
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,5282.02,2025-05-07 22:00:00
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,5746.14,2025-05-07 22:00:00
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,6098.12,2025-05-07 22:00:00
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,6322.01,2025-05-07 22:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,0,0,0,0,0,0,0,0,0,0,...,0,0,0,2,0,0,0,0,JC108,2025-05-07 22:00:00
91,1,0,0,0,0,0,0,0,0,2,...,2,10,8,13,14,11,6,1,JC109,2025-05-07 22:00:00
92,0,0,1,0,0,0,1,0,1,2,...,1,2,5,5,4,1,5,4,JC110,2025-05-07 22:00:00
93,3,0,0,0,0,0,0,1,8,2,...,3,12,15,37,26,17,4,3,JC115,2025-05-07 22:00:00


In [14]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

2025-05-08 17:29:57,902 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-08 17:29:57,910 INFO: Initializing external client
2025-05-08 17:29:57,911 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-08 17:29:58,793 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1213633


Downloading: 0.000%|          | 0/320762 elapsed<00:00 remaining<?

In [15]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

In [16]:
predictions

,pickup_location_id,pickup_hour,predicted_demand
0,5187.03,2025-05-07 22:00:00,0.0
1,5282.02,2025-05-07 22:00:00,0.0
2,5746.14,2025-05-07 22:00:00,0.0
3,6098.12,2025-05-07 22:00:00,0.0
4,6322.01,2025-05-07 22:00:00,0.0
...,...,...,...
90,JC108,2025-05-07 22:00:00,0.0
91,JC109,2025-05-07 22:00:00,1.0
92,JC110,2025-05-07 22:00:00,1.0
93,JC115,2025-05-07 22:00:00,2.0


In [17]:
predictions.sort_values("predicted_demand", ascending=False).head(10)["pickup_location_id"].values

array(['HB105', 'HB611', 'HB102', 'JC032', 'HB601', 'HB602', 'HB608',
       'JC003', 'JC008', 'JC009'], dtype=object)